# 02 - Preprocessing and Feature Engineering

**Project**: Preventing Injuries in Triathletes with Machine Learning using Wearable Sensor Data  
**Notebook**: 02 - Preprocessing and Feature Engineering
**Author**: Leonardo Rossi  

---

**Objectives**:
- Cleaning and standardizing columns
- Engineering domain-specific features (e.g. HRV ratios, training deviations, ACWR)
- Creating rolling averages and binary flags
- Preparing the final dataset for machine learning
- Splitting this dataset in training and testing parts


In [255]:
# Imports
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import ast
from sklearn.preprocessing import OneHotEncoder, OrdinalEncoder
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import GroupShuffleSplit

# Display settings
pd.set_option('display.max_columns', None)
sns.set(style="whitegrid")


## Load Merged Data

We load the merged dataset prepared during EDA.


In [256]:
# Load from disk and make a copy to work with
eda_merged = pd.read_pickle("../simulated_data/processed/merged.pkl")
merged = eda_merged.copy()

# Quick check
print(f"Shape: {merged.shape}")
merged.head()

Shape: (275527, 92)


,athlete_id,gender,age,height_cm,weight_kg,genetic_factor,hrv_baseline,hrv_range,max_hr,resting_hr_norm,lthr,hr_zones,vo2max,running_threshold_pace,ftp,css,training_experience,weekly_training_hours,recovery_rate,lifestyle,sleep_time_norm,sleep_quality_norm,nutrition_factor,stress_factor,smoking_factor,drinking_factor,date,resting_hr_daily,hrv,sleep_hours,deep_sleep,light_sleep,rem_sleep,sleep_quality_daily,body_battery_morning,stress,body_battery_evening,planned_tss,actual_tss,injury,swim_duration_minutes,swim_hr_zones,swim_power_zones,swim_tss,swim_intensity_factor,swim_distance_km,swim_avg_speed_kph,swim_avg_hr,swim_max_hr,swim_avg_power,swim_normalized_power,swim_work_kilojoules,swim_elevation_gain,bike_duration_minutes,bike_hr_zones,bike_power_zones,bike_tss,bike_intensity_factor,bike_distance_km,bike_avg_speed_kph,bike_avg_hr,bike_max_hr,bike_avg_power,bike_normalized_power,bike_work_kilojoules,bike_elevation_gain,run_duration_minutes,run_hr_zones,run_power_zones,run_tss,run_intensity_factor,run_distance_km,run_avg_speed_kph,run_avg_hr,run_max_hr,run_avg_power,run_normalized_power,run_work_kilojoules,run_elevation_gain,strength_duration_minutes,strength_hr_zones,strength_power_zones,strength_tss,strength_intensity_factor,strength_distance_km,strength_avg_speed_kph,strength_avg_hr,strength_max_hr,strength_avg_power,strength_normalized_power,strength_work_kilojoules,strength_elevation_gain
0,97edad40-801f-42c8-98fc-d0fce938c24f,female,35,164,57.7,0.91,97.5,"(np.float64(82.9), np.float64(112.1))",187.5,44.9,188,"{'Z1': (np.float64(67.39847066423759), 150.4),...",68.9,4.28,210.3,94.1,4,13.6,1.09,Highly Disciplined Athlete,7.733992,0.905808,0.986618,0.141615,0.0,0.060112,2024-01-01,43.518730,103.162006,8.743521,1.348662,5.451481,1.943378,0.921,100.0,4.7,46.3,75,75.0,0,56.0,"{'Z1': 15.178571428571427, 'Z2': 4.01785714285...",None,75.0,0.9,3.629686,3.888949,174.0,188.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,0.0,0.00,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,0.0,0.00,0.00,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,97edad40-801f-42c8-98fc-d0fce938c24f,female,35,164,57.7,0.91,97.5,"(np.float64(82.9), np.float64(112.1))",187.5,44.9,188,"{'Z1': (np.float64(67.39847066423759), 150.4),...",68.9,4.28,210.3,94.1,4,13.6,1.09,Highly Disciplined Athlete,7.733992,0.905808,0.986618,0.141615,0.0,0.060112,2024-01-02,41.774490,103.266783,8.068680,1.259958,5.004577,1.804145,0.921,100.0,6.9,41.2,103,103.0,0,0.0,NaN,NaN,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,154.0,"{'Z1': 19.967532467532468, 'Z2': 42.2077922077...","{'Z1': 31.006493506493506, 'Z2': 68.9935064935...",103.0,0.63,74.69,29.1,159.0,179.0,118.0,119.0,1090.0,373.0,0.0,NaN,NaN,0.0,0.00,0.00,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,97edad40-801f-42c8-98fc-d0fce938c24f,female,35,164,57.7,0.91,97.5,"(np.float64(82.9), np.float64(112.1))",187.5,44.9,188,"{'Z1': (np.float64(67.39847066423759), 150.4),...",68.9,4.28,210.3,94.1,4,13.6,1.09,Highly Disciplined Athlete,7.733992,0.905808,0.986618,0.141615,0.0,0.060112,2024-01-04,45.185704,101.138027,7.794236,1.301622,4.699527,1.793086,0.841,100.0,10.0,42.5,93,93.0,0,70.0,"{'Z1': 99.64285714285714, 'Z2': 0.0, 'Z3': 0.0...",None,29.0,0.5,4.401132,3.772399,122.0,133.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,0.0,0.00,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,77.0,"{'Z1': 21.753246753246753, 'Z2': 78.2467532467...",None,64.0,0.71,12.85,10.01,153.0,168.0,0.0,0.0,0.0,128.0,0.0,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,97edad40-801f-42c8-98fc-d0fce938c24f,female,35,164,57.7,0.91,97.5,"(np.float64(82.9), np.float64(112.1))",187.5,44.9,188,"{'Z1': (np.float64(67.39847066423759), 150.4),...",68.9,4.28,210.3,94.1,4,13.6,1.09,Highly Disciplined Athlete,7.733992,0.905808,0.986618,0.141615,0.0,0.060112,2024-01-06,44.715721,101.155273,8.064840,1.389732,4.794163,1.880946,0.948,100.0,9.6,41.6,111,111.0,0,67.0,"{'Z1': 15.298507462686567, 'Z2': 3.35820895522...",None,111.0,1.0,4.210747,3.770818,174.

## Basic Preprocessing

Since this dataset was synthetically created this step does not involve much, missing data is minimal and only exists for sport specific variables (e.g. cycling power on running workouts is naturally missing). Also no duplicates exist. We will do the following:

- Convert dates
- Handle missing values
- drop hrv_range as this is simply 85% - 115% of baseline
- drop athlete hr_zones as they are simply fixed percentages of lthr
- flatten the zone dictionaries into individual columns


In [257]:
# Convert to datetime
merged['date'] = pd.to_datetime(merged['date'])

# Fill or impute missing values
merged.fillna(0, inplace=True)

merged.drop(['hrv_range', 'hr_zones'], axis=1, inplace=True)

/var/folders/lx/jx57qljj5n74_nkl5qm8bqcr0000gn/T/ipykernel_1574/913812685.py:5: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  merged.fillna(0, inplace=True)


In [258]:
# List of dictionary columns to flatten
dict_columns = [
    'bike_hr_zones', 'run_hr_zones', 
    'bike_power_zones', 'run_power_zones',
    'swim_hr_zones', 'swim_power_zones',
    'strength_hr_zones', 'strength_power_zones'
]

# Function to safely convert to dictionary
def safe_convert_to_dict(x):
    if pd.isna(x):
        return {}
    if isinstance(x, dict):
        return x
    if isinstance(x, str):
        try:
            return ast.literal_eval(x)
        except:
            return {}
    return {}

# Function to flatten the dictionary
def flatten_dict_column(df, col_name):
    # Check if column exists in the dataframe
    if col_name not in df.columns:
        return df
    
    # Safely convert to dictionary
    df[col_name] = df[col_name].apply(safe_convert_to_dict)
    
    # Create new column names
    prefix = col_name + '_'
    
    # Get all possible keys across all dictionaries
    all_keys = set()
    for d in df[col_name].dropna():
        if isinstance(d, dict):
            all_keys.update(d.keys())
    
    # Create a new column for each key
    for key in sorted(all_keys):  # Sort keys to ensure consistent order
        new_col = prefix + key
        df[new_col] = df[col_name].apply(lambda x: x.get(key, 0) if isinstance(x, dict) else 0)
    
    return df

# Apply the function to each dictionary column
for col in dict_columns:
    if col in merged.columns:  # Only process columns that exist
        merged = flatten_dict_column(merged, col)

# Optionally, drop the original dictionary columns
merged = merged.drop(columns=[col for col in dict_columns if col in merged.columns])

merged.head()

,athlete_id,gender,age,height_cm,weight_kg,genetic_factor,hrv_baseline,max_hr,resting_hr_norm,lthr,vo2max,running_threshold_pace,ftp,css,training_experience,weekly_training_hours,recovery_rate,lifestyle,sleep_time_norm,sleep_quality_norm,nutrition_factor,stress_factor,smoking_factor,drinking_factor,date,resting_hr_daily,hrv,sleep_hours,deep_sleep,light_sleep,rem_sleep,sleep_quality_daily,body_battery_morning,stress,body_battery_evening,planned_tss,actual_tss,injury,swim_duration_minutes,swim_tss,swim_intensity_factor,swim_distance_km,swim_avg_speed_kph,swim_avg_hr,swim_max_hr,swim_avg_power,swim_normalized_power,swim_work_kilojoules,swim_elevation_gain,bike_duration_minutes,bike_tss,bike_intensity_factor,bike_distance_km,bike_avg_speed_kph,bike_avg_hr,bike_max_hr,bike_avg_power,bike_normalized_power,bike_work_kilojoules,bike_elevation_gain,run_duration_minutes,run_tss,run_intensity_factor,run_distance_km,run_avg_speed_kph,run_avg_hr,run_max_hr,run_avg_power,run_normalized_power,run_work_kilojoules,run_elevation_gain,strength_duration_minutes,strength_tss,strength_intensity_factor,strength_distance_km,strength_avg_speed_kph,strength_avg_hr,strength_max_hr,strength_avg_power,strength_normalized_power,strength_work_kilojoules,strength_elevation_gain,bike_hr_zones_Z1,bike_hr_zones_Z2,bike_hr_zones_Z3,bike_hr_zones_Z4,bike_hr_zones_Z5,bike_hr_zones_Z6,run_hr_zones_Z1,run_hr_zones_Z2,run_hr_zones_Z3,run_hr_zones_Z4,run_hr_zones_Z5,run_hr_zones_Z6,bike_power_zones_Z1,bike_power_zones_Z2,bike_power_zones_Z3,bike_power_zones_Z4,bike_power_zones_Z5,bike_power_zones_Z6,bike_power_zones_Z7,swim_hr_zones_Z1,swim_hr_zones_Z2,swim_hr_zones_Z3,swim_hr_zones_Z4,swim_hr_zones_Z5,swim_hr_zones_Z6,strength_hr_zones_Z1,strength_hr_zones_Z2,strength_hr_zones_Z3,strength_hr_zones_Z4,strength_hr_zones_Z5,strength_hr_zones_Z6
0,97edad40-801f-42c8-98fc-d0fce938c24f,female,35,164,57.7,0.91,97.5,187.5,44.9,188,68.9,4.28,210.3,94.1,4,13.6,1.09,Highly Disciplined Athlete,7.733992,0.905808,0.986618,0.141615,0.0,0.060112,2024-01-01,43.518730,103.162006,8.743521,1.348662,5.451481,1.943378,0.921,100.0,4.7,46.3,75,75.0,0,56.0,75.0,0.9,3.629686,3.888949,174.0,188.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.00,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,15.178571,4.017857,1.785714,78.571429,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,97edad40-801f-42c8-98fc-d0fce938c24f,female,35,164,57.7,0.91,97.5,187.5,44.9,188,68.9,4.28,210.3,94.1,4,13.6,1.09,Highly Disciplined Athlete,7.733992,0.905808,0.986618,0.141615,0.0,0.060112,2024-01-02,41.774490,103.266783,8.068680,1.259958,5.004577,1.804145,0.921,100.0,6.9,41.2,103,103.0,0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,154.0,103.0,0.63,74.69,29.1,159.0,179.0,118.0,119.0,1090.0,373.0,0.0,0.0,0.00,0.00,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,19.967532,42.207792,37.5,0.324675,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,31.006494,68.993506,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,97edad40-801f-42c8-98fc-d0fce938c24f,female,35,164,57.7,0.91,97.5,187.5,44.9,188,68.9,4.28,210.3,94.1,4,13.6,1.09,Highly Disciplined Athlete,7.733992,0.905808,0.986618,0.141615,0.0,0.060112,2024-01-04,45.185704,101.138027,7.794236,1.301622,4.699527,1.793086,0.841,100.0,10.0,42.5,93,93.0,0,70.0,29.0,0.5,4.401132,3.772399,122.0,133.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,77.0,64.0,0.71,12.85,10.01,153.0,168.0,0.0,0.0,0.0,128.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.0,21.753247,78.246753,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,99.642857,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,97edad40-801f-42c8-98fc-d0fce938c24f,female,35,164,57.7,0.91,97.5,187.5,44.9,188,68.9,4.28,210.3,94.1,4,13.6,1.09

Notice that some features are sport specific, (such as power, normalized power and work in kilojoules) and are therefore always zero for all other sports. We will drop these as they don't add any value and add unneccesary complexity.

In [259]:
null_columns = ['run_avg_power', 'run_normalized_power', 'run_work_kilojoules',
                'swim_avg_power', 'swim_normalized_power', 'swim_work_kilojoules', 
                'swim_elevation_gain', 'strength_avg_power', 'strength_normalized_power', 
                'strength_work_kilojoules', 'strength_elevation_gain', 'strength_distance_km', 
                'strength_avg_speed_kph']

merged.drop(null_columns, axis=1, inplace=True)

merged.head()

,athlete_id,gender,age,height_cm,weight_kg,genetic_factor,hrv_baseline,max_hr,resting_hr_norm,lthr,vo2max,running_threshold_pace,ftp,css,training_experience,weekly_training_hours,recovery_rate,lifestyle,sleep_time_norm,sleep_quality_norm,nutrition_factor,stress_factor,smoking_factor,drinking_factor,date,resting_hr_daily,hrv,sleep_hours,deep_sleep,light_sleep,rem_sleep,sleep_quality_daily,body_battery_morning,stress,body_battery_evening,planned_tss,actual_tss,injury,swim_duration_minutes,swim_tss,swim_intensity_factor,swim_distance_km,swim_avg_speed_kph,swim_avg_hr,swim_max_hr,bike_duration_minutes,bike_tss,bike_intensity_factor,bike_distance_km,bike_avg_speed_kph,bike_avg_hr,bike_max_hr,bike_avg_power,bike_normalized_power,bike_work_kilojoules,bike_elevation_gain,run_duration_minutes,run_tss,run_intensity_factor,run_distance_km,run_avg_speed_kph,run_avg_hr,run_max_hr,run_elevation_gain,strength_duration_minutes,strength_tss,strength_intensity_factor,strength_avg_hr,strength_max_hr,bike_hr_zones_Z1,bike_hr_zones_Z2,bike_hr_zones_Z3,bike_hr_zones_Z4,bike_hr_zones_Z5,bike_hr_zones_Z6,run_hr_zones_Z1,run_hr_zones_Z2,run_hr_zones_Z3,run_hr_zones_Z4,run_hr_zones_Z5,run_hr_zones_Z6,bike_power_zones_Z1,bike_power_zones_Z2,bike_power_zones_Z3,bike_power_zones_Z4,bike_power_zones_Z5,bike_power_zones_Z6,bike_power_zones_Z7,swim_hr_zones_Z1,swim_hr_zones_Z2,swim_hr_zones_Z3,swim_hr_zones_Z4,swim_hr_zones_Z5,swim_hr_zones_Z6,strength_hr_zones_Z1,strength_hr_zones_Z2,strength_hr_zones_Z3,strength_hr_zones_Z4,strength_hr_zones_Z5,strength_hr_zones_Z6
0,97edad40-801f-42c8-98fc-d0fce938c24f,female,35,164,57.7,0.91,97.5,187.5,44.9,188,68.9,4.28,210.3,94.1,4,13.6,1.09,Highly Disciplined Athlete,7.733992,0.905808,0.986618,0.141615,0.0,0.060112,2024-01-01,43.518730,103.162006,8.743521,1.348662,5.451481,1.943378,0.921,100.0,4.7,46.3,75,75.0,0,56.0,75.0,0.9,3.629686,3.888949,174.0,188.0,0.0,0.0,0.00,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.00,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,15.178571,4.017857,1.785714,78.571429,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,97edad40-801f-42c8-98fc-d0fce938c24f,female,35,164,57.7,0.91,97.5,187.5,44.9,188,68.9,4.28,210.3,94.1,4,13.6,1.09,Highly Disciplined Athlete,7.733992,0.905808,0.986618,0.141615,0.0,0.060112,2024-01-02,41.774490,103.266783,8.068680,1.259958,5.004577,1.804145,0.921,100.0,6.9,41.2,103,103.0,0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,154.0,103.0,0.63,74.69,29.1,159.0,179.0,118.0,119.0,1090.0,373.0,0.0,0.0,0.00,0.00,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,19.967532,42.207792,37.5,0.324675,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,31.006494,68.993506,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,97edad40-801f-42c8-98fc-d0fce938c24f,female,35,164,57.7,0.91,97.5,187.5,44.9,188,68.9,4.28,210.3,94.1,4,13.6,1.09,Highly Disciplined Athlete,7.733992,0.905808,0.986618,0.141615,0.0,0.060112,2024-01-04,45.185704,101.138027,7.794236,1.301622,4.699527,1.793086,0.841,100.0,10.0,42.5,93,93.0,0,70.0,29.0,0.5,4.401132,3.772399,122.0,133.0,0.0,0.0,0.00,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,77.0,64.0,0.71,12.85,10.01,153.0,168.0,128.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.0,21.753247,78.246753,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,99.642857,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,97edad40-801f-42c8-98fc-d0fce938c24f,female,35,164,57.7,0.91,97.5,187.5,44.9,188,68.9,4.28,210.3,94.1,4,13.6,1.09,Highly Disciplined Athlete,7.733992,0.905808,0.986618,0.141615,0.0,0.060112,2024-01-06,44.715721,101.155273,8.064840,1.389732,4.794163,1.880946,0.948,100.0,9.6,41.6,111,111.0,0,67.0,111.0,1.0,4.210747,3.770818,174.0,188.0,0.0,0.0,0.00,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.00,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0

## Label Engineering

As we saw during exploratory data analysis the fact that there is a single binary injury label, meaning that both first injury days and following recovery days are labeled positiveley is misleading. For that reason we transform the raw injury labels into more meaningful labels:

### `engineer_injury_labels`
Transforms basic injury indicators (`0` / `1`) into more detailed labels:

- **`injury_onset`**:  
  Marks the **first day** of an injury occurrence.

- **`recovery_day`**:  
  Marks days **during injury recovery**.

- **`pre_injury`**:  
  Marks days **leading up to an injury** (within a defined prediction window).

- **`injury_state`** *(multi-class label)*:  
  - `0` = Healthy  
  - `1` = Pre-injury  
  - `2` = Injury onset  
  - `3` = Recovery


### `create_prediction_targets`
Creates a target variable for machine learning models to predict:

- **`will_get_injured`**:  
  Indicates whether an injury **will occur within a specified future window**.


### `create_time_to_injury_feature`
Creates a feature representing the time until the next injury:

- **`time_to_injury`**:  
  Number of **days until the next injury onset** (capped at a maximum threshold).

In [260]:
def engineer_injury_labels(df, prediction_window=7):
    """
    Transform raw injury labels into more meaningful labels for machine learning.
    
    Parameters:
    -----------
    df : pandas DataFrame
        DataFrame containing daily athlete data with 'injury' column (1=injured, 0=not injured)
    prediction_window : int
        Number of days before injury to label as "pre-injury" state
        
    Returns:
    --------
    DataFrame with new label columns added
    """
    # Make a copy to avoid modifying the original
    result_df = df.copy()
    
    # Sort by athlete ID and date to ensure correct temporal order
    result_df = result_df.sort_values(['athlete_id', 'date'])
    
    # Create new label columns
    result_df['injury_onset'] = 0  # Will be 1 only on the first day of injury
    result_df['recovery_day'] = 0  # Will be 1 for days in recovery (after onset)
    result_df['pre_injury'] = 0    # Will be 1 for days leading up to injury
    result_df['injury_state'] = 0  # Multi-class: 0=healthy, 1=pre-injury, 2=onset, 3=recovery
    
    # Process each athlete separately
    for athlete_id in result_df['athlete_id'].unique():
        athlete_data = result_df[result_df['athlete_id'] == athlete_id].copy()
        
        # Get indices for this athlete's data in the main dataframe
        athlete_indices = athlete_data.index
        
        # Find injury onset days (where injury=1 but previous day injury=0)
        injury_indices = []
        
        for i in range(1, len(athlete_indices)):
            prev_idx = athlete_indices[i-1]
            curr_idx = athlete_indices[i]
            
            if (result_df.loc[curr_idx, 'injury'] == 1 and 
                result_df.loc[prev_idx, 'injury'] == 0):
                injury_indices.append(curr_idx)
        
        # First day might be an injury onset too
        if len(athlete_indices) > 0 and result_df.loc[athlete_indices[0], 'injury'] == 1:
            injury_indices.append(athlete_indices[0])
        
        # Mark injury onset days
        result_df.loc[injury_indices, 'injury_onset'] = 1
        
        # For each injury onset, label the days leading up to it as pre-injury
        for onset_idx in injury_indices:
            # Find this onset's position in the athlete's timeline
            onset_position = list(athlete_indices).index(onset_idx)
            
            # Label previous 'prediction_window' days as pre-injury, if they exist
            for i in range(1, prediction_window + 1):
                if onset_position - i >= 0:  # Ensure we don't go before the first day
                    pre_injury_idx = athlete_indices[onset_position - i]
                    # Only label as pre-injury if not already injured
                    if result_df.loc[pre_injury_idx, 'injury'] == 0:
                        result_df.loc[pre_injury_idx, 'pre_injury'] = 1
        
        # Mark recovery days (injured but not onset)
        recovery_indices = [idx for idx in athlete_indices 
                           if result_df.loc[idx, 'injury'] == 1 
                           and idx not in injury_indices]
        result_df.loc[recovery_indices, 'recovery_day'] = 1
        
        # Create multi-class injury state column
        # Priority: onset > recovery > pre_injury > healthy
        result_df.loc[result_df['injury_onset'] == 1, 'injury_state'] = 2
        result_df.loc[(result_df['recovery_day'] == 1) & (result_df['injury_state'] == 0), 'injury_state'] = 3
        result_df.loc[(result_df['pre_injury'] == 1) & (result_df['injury_state'] == 0), 'injury_state'] = 1
        # Healthy stays as 0
    
    return result_df

def create_prediction_targets(df, predict_window=7):
    """
    Create binary target columns for predicting:
    1. Will injury occur in the next X days
    2. Is this the onset of an injury
    
    Parameters:
    -----------
    df : pandas DataFrame
        DataFrame with injury data
    predict_window : int
        Window for future injury prediction (we take 7 days as this emerged as most meaningful during EDA)
        
    Returns:
    --------
    DataFrame with prediction target columns
    """
    result_df = df.copy()
    
    # Initialize the target columns
    result_df['will_get_injured'] = 0
    
    # Process each athlete separately
    for athlete_id in result_df['athlete_id'].unique():
        athlete_mask = result_df['athlete_id'] == athlete_id
        athlete_data = result_df[athlete_mask].copy()
        
        # For each day, check if injury will occur within prediction window
        for i in range(len(athlete_data) - 1):
            # Look ahead up to prediction_window days
            look_ahead = min(predict_window, len(athlete_data) - i - 1)
            
            # If there's an injury onset in the look-ahead window, mark current day
            if athlete_data['injury_onset'].iloc[i+1:i+1+look_ahead].sum() > 0:
                result_df.loc[athlete_data.index[i], 'will_get_injured'] = 1
    
    return result_df

def create_time_to_injury_feature(df, max_days=28):
    """
    Create a feature representing days until next injury (up to max_days)
    
    Parameters:
    -----------
    df : pandas DataFrame
        DataFrame with injury data
    max_days : int
        Maximum number of days to look ahead
        
    Returns:
    --------
    DataFrame with time_to_injury column
    """
    result_df = df.copy()
    result_df['time_to_injury'] = max_days + 1  # Default: more than max_days
    
    # Process each athlete separately
    for athlete_id in result_df['athlete_id'].unique():
        athlete_data = result_df[result_df['athlete_id'] == athlete_id].copy()
        
        # Find indices of injury onsets for this athlete
        onset_indices = athlete_data[athlete_data['injury_onset'] == 1].index
        
        # For each day, calculate days until next injury
        for idx in athlete_data.index:
            # Find the next injury onset after this day
            future_onsets = [i for i in onset_indices if i > idx]
            
            if future_onsets:
                next_onset = future_onsets[0]
                days_until = athlete_data.index.get_loc(next_onset) - athlete_data.index.get_loc(idx)
                
                # Only record if within max_days
                if days_until <= max_days:
                    result_df.loc[idx, 'time_to_injury'] = days_until
    
    return result_df

# Step 1: Engineer basic injury labels
df_labeled = engineer_injury_labels(merged, prediction_window=7)

# Step 2: Create prediction targets
df_targets = create_prediction_targets(df_labeled, predict_window=7)

# Step 3: Add time-to-injury as a feature or alternative target
df_final = create_time_to_injury_feature(df_targets, max_days=28)

Lastly, before feature engineering we want to split the data into features and target (injury label)

In [269]:
X = df_final.drop(['injury', 'injury_onset', 'recovery_day', 'pre_injury', 
                  'injury_state', 'will_get_injured', 'time_to_injury'], axis=1)
y = df_final['will_get_injured'] 

## Feature Engineering

Building on our EDA insights, we now engineer features that capture critical patterns preceding injury events. These features are tailored to reflect physiological stress, recovery, and workload balance, as highlighted in our analysis:

- **Personalized Baseline Metrics:**  
  Individualized z-scores for recovery metrics (e.g., HRV, sleep, resting HR), emphasizing the critical 7-day pre-injury window where deviations from personal norms were most pronounced.

- **Stress Response Features:**  
  Since stress exhibited the strongest pre-injury signal (up to +3.21σ), we include rolling volatility, rate-of-change measures, exponential growth detection, and stress-to-recovery ratios (e.g., stress-to-HRV, stress-to-sleep).

- **Sleep Quality Indicators:**  
  Multi-dimensional tracking of sleep quality (overall score, deep sleep, REM), incorporating baseline deviation, trend slope, and sustained sub-baseline days — reflecting the consistent sleep deterioration seen before injury.

- **Distribution Shape Tracking:**  
  Features measuring the spread and consistency of key metrics (e.g., coefficient of variation, rolling std/mean) based on violin plot findings that showed narrowing distributions before injury.

- **Recovery-Load Interaction:**  
  Interaction features linking acute:chronic workload ratio (ACWR) with recovery metrics (HRV, sleep, resting HR), flagging cases where load appears manageable but recovery is insufficient.

- **Multi-Window Prediction Features:**  
  Inclusion of short- (1d), medium- (7d), and long-term (28d, 42d) rolling features, exponential moving averages, and slope/acceleration metrics to capture varying temporal risk signals.

- **Composite & Risk Scores:**  
  Generation of recovery and stress risk scores using weighted combinations of volatility, deviation, and duration-based indicators, enabling easier aggregation and threshold-based monitoring.

In [270]:
def engineer_features(X):
    """
    Create comprehensive features for athlete performance monitoring and risk analysis.
    
    Parameters:
    -----------
    X : pandas DataFrame
        Input data with raw athlete metrics
        
    Returns:
    --------
    X : pandas DataFrame
        DataFrame with engineered features
    """
    
    # Helper function to safely calculate ratios and prevent infinities
    def safe_ratio(numerator, denominator, default=0, lower_clip=None):
        result = np.where(denominator != 0, numerator / denominator, default)
        result = pd.Series(result).replace([np.inf, -np.inf], default)
        if lower_clip is not None:
            result = result.clip(lower=lower_clip)
        return result
    
    # Helper function for calculating rolling metrics by athlete
    def rolling_by_athlete(df, column, window, function, min_periods=1):
        return df.groupby('athlete_id')[column].transform(
            lambda x: x.rolling(window=window, min_periods=min_periods).apply(function)
        )
    
    # Helper function for consecutive day counters
    def consecutive_day_counter(df, column):
        return (
            df.groupby('athlete_id')[column]
            .transform(lambda x: x.groupby((x != x.shift()).cumsum()).cumcount() + 1)
            * df[column]
        )
    
    # ===== HRV-Based Features =====
    # Basic HRV metrics
    X['hrv_ratio'] = X['hrv'] / X['hrv_baseline']
    X['hrv_zone'] = pd.cut(
        X['hrv_ratio'],
        bins=[0, 0.851, 1.15, np.inf],
        labels=['Below Normal', 'Normal', 'Above Normal']
    )
    X['low_hrv_risk'] = (X['hrv_ratio'] < 0.85).astype(int)
    X['consecutive_low_hrv'] = consecutive_day_counter(X, 'low_hrv_risk')
    
    # HRV trends and volatility
    X['hrv_7d_slope'] = rolling_by_athlete(
        X, 'hrv_ratio', 7, 
        lambda y: np.polyfit(range(len(y)), y, 1)[0] if len(y) > 2 else np.nan,
        min_periods=3
    )
    X['hrv_volatility'] = rolling_by_athlete(X, 'hrv', 7, np.std, min_periods=3)
    X['hrv_volatility_risk'] = X['hrv_volatility'] * 0.15
    
    # ===== Resting Heart Rate Features =====
    X['resting_hr_ratio'] = X['resting_hr_daily'] / X['resting_hr_norm']
    X['rhr_risk'] = np.where(X['resting_hr_ratio'] > 1.2,
                            (X['resting_hr_ratio'] - 1.2) * 0.15,
                            0)
    
    # ===== Sleep Features =====
    X['poor_sleep'] = (X['sleep_quality_daily'] < 0.6).astype(int)
    
    # Create sleep debt metrics if columns exist
    if 'sleep_hours' in X.columns and 'sleep_time_norm' in X.columns:
        X['sleep_debt'] = (X['sleep_time_norm'] - X['sleep_hours']).clip(lower=0)
        X['sleep_hours_risk'] = np.where(X['sleep_debt'] > 2,
                                        (X['sleep_debt'] - 2) * 0.02, 
                                        0)
    
    # ===== Training Load Features =====
    # Training load deviations
    X['tss_deviation'] = X['actual_tss'] - X['planned_tss']
    X['tss_deviation_pct'] = safe_ratio(X['tss_deviation'], X['planned_tss'], default=1000) * 100
    
    # Training load categorization
    X['high_tss_day'] = (X['actual_tss'] > X['actual_tss'].quantile(0.85)).astype(int)
    X['consecutive_high_tss'] = consecutive_day_counter(X, 'high_tss_day')
    X['training_adherence'] = pd.cut(
        X['tss_deviation_pct'],
        bins=[-np.inf, -15, 15, np.inf],
        labels=['Undertraining', 'On Target', 'Overtraining']
    )
    
    # ===== Acute/Chronic Workload Features =====
    # Calculate rolling averages and sums
    X['tss_7d_avg'] = rolling_by_athlete(X, 'actual_tss', 7, np.mean)
    X['acute_load'] = rolling_by_athlete(X, 'actual_tss', 7, np.sum)
    X['chronic_load'] = rolling_by_athlete(X, 'actual_tss', 28, np.mean) * 7
    
    # Calculate ACWR and risk zones
    X['acwr'] = safe_ratio(X['acute_load'], X['chronic_load'])
    X['acwr_risk'] = pd.cut(
        X['acwr'],
        bins=[0, 0.8, 1.3, 1.5, np.inf],
        labels=['Too Low', 'Optimal', 'Danger Zone', 'High Risk']
    )
    
    # ACWR volatility
    X['acwr_volatility'] = rolling_by_athlete(X, 'acwr', 7, np.std, min_periods=3)
    X['acwr_volatility_risk'] = X['acwr_volatility'] * 0.2
    
    # ===== Training Monotony & Strain =====
    X['tss_7d_std'] = rolling_by_athlete(X, 'actual_tss', 7, np.std, min_periods=2)
    X['training_monotony'] = safe_ratio(X['tss_7d_avg'], X['tss_7d_std'])
    X['training_strain'] = X['tss_7d_avg'] * 7 * X['training_monotony']
    
    # ===== Week-to-Week Load Change (Ramp Rate) =====
    X['previous_week_load'] = X.groupby('athlete_id')['acute_load'].shift(7).fillna(0)
    X['week_to_week_change'] = safe_ratio(
        (X['acute_load'] - X['previous_week_load']), 
        X['previous_week_load']
    ) * 100
    
    X['ramp_rate_risk'] = pd.cut(
        X['week_to_week_change'],
        bins=[-np.inf, -5, 10, 20, np.inf],
        labels=['Decreasing', 'Optimal', 'Warning', 'High Risk']
    )
    
    # ===== Advanced Load Metrics =====
    # Add longer-term metrics
    X['chronic_load_42d'] = rolling_by_athlete(X, 'actual_tss', 42, np.mean) * 7
    
    # Exponentially weighted metrics
    X['ewma_acute'] = X.groupby('athlete_id')['actual_tss'].transform(
        lambda x: x.ewm(span=7, min_periods=1).mean() * 7
    )
    X['ewma_chronic'] = X.groupby('athlete_id')['actual_tss'].transform(
        lambda x: x.ewm(span=28, min_periods=1).mean() * 28
    )
    X['ewma_acwr'] = safe_ratio(X['ewma_acute'], X['ewma_chronic'])
    
    # ===== Interaction Features =====
    # Create interaction features
    X['load_recovery_interaction'] = X['acwr'] * safe_ratio(1, X['hrv_ratio'], lower_clip=0.5)
    X['sleep_hrv_compound'] = X['poor_sleep'] * X['low_hrv_risk']
    
    # Composite scores
    X['recovery_composite'] = (
        (X['hrv_ratio'] * 0.4) + 
        ((1 - X['poor_sleep']) * 0.3) + 
        ((1 - X.get('rhr_risk', 0)) * 0.3)
    )
    
    # Risk estimation
    X['acute_risk_estimate'] = (
        X.get('fatigue_risk', 0) * 0.15 +
        X.get('hrv_critical_risk', 0) * 0.1 +
        X.get('rhr_risk', 0) * 0.05 +
        X.get('poor_sleep', 0) * 0.05 +
        X.get('acwr_high_risk', 0) * 0.15
    )
    
    X['high_risk_multiplier'] = 1.0 + np.maximum(0, (X['acute_risk_estimate'] - 0.3) * 2.0)
    
    # ===== Training Pattern Detection =====
    X['tss_relative_change'] = X.groupby('athlete_id')['actual_tss'].pct_change()
    X['tss_spike'] = (X['tss_relative_change'] > 0.5).astype(int)
    
    # Detect taper-spike patterns
    X['tss_3d_mean'] = rolling_by_athlete(X, 'actual_tss', 3, np.mean)
    X['tss_prev_10d_mean'] = X.groupby('athlete_id')['actual_tss'].transform(
        lambda x: x.shift(3).rolling(10).mean()
    )
    X['taper_spike_pattern'] = (
        (X['tss_3d_mean'] < 0.7 * X['tss_prev_10d_mean']) & 
        (X['tss_3d_mean'].shift(-3) > 1.3 * X['tss_3d_mean'])
    ).astype(int)
    
    # ===== Recovery Adequacy Features =====
    X['recovery_adequacy'] = safe_ratio(X['hrv_ratio'], X['acwr'], lower_clip=0.5)
    X['sleep_to_load_ratio'] = safe_ratio(X['sleep_quality_daily'], X['tss_7d_avg'], lower_clip=1)
    
    # Recovery window detection
    X['recovery_window_after_high_load'] = (
        (X['high_tss_day'].shift(1) == 1) & 
        (X['actual_tss'] < X['actual_tss'].quantile(0.3))
    ).astype(int)
    
    # ===== Polynomial and Complex Interactions =====
    X['acwr_squared'] = X['acwr'] ** 2
    X['acwr_hrv_interaction'] = X['acwr'] * (2 - X['hrv_ratio'].clip(upper=2))
    
    if 'sleep_hours_risk' in X.columns:
        X['acwr_sleep_interaction'] = X['acwr'] * (2 - X['sleep_hours_risk'].clip(upper=2))
    
    if 'baseline_injury_risk' in X.columns:
        X['combined_risk_interaction'] = (
            X['acwr'].clip(lower=0.5) * 
            safe_ratio(1, X['recovery_composite'], lower_clip=0.5) * 
            X['baseline_injury_risk']
        )
    else:
        X['combined_risk_interaction'] = (
            X['acwr'].clip(lower=0.5) * 
            safe_ratio(1, X['recovery_composite'], lower_clip=0.5)
        )
    
    # ===== Rate of Change Features =====
    key_metrics = ['acute_load', 'chronic_load', 'hrv_ratio', 'sleep_quality_daily']
    
    # Add 'recovery_debt' if it exists in columns
    if 'recovery_debt' in X.columns:
        key_metrics.append('recovery_debt')
    
    for feature in key_metrics:
        if feature in X.columns:
            X[f'{feature}_change_1d'] = X.groupby('athlete_id')[feature].diff()
            X[f'{feature}_change_7d'] = X.groupby('athlete_id')[feature].diff(7)
            X[f'{feature}_acceleration'] = X.groupby('athlete_id')[f'{feature}_change_1d'].diff()
    
    # ===== Statistical Features =====
    # Coefficient of variation
    for feature in ['actual_tss', 'hrv', 'sleep_hours']:
        if feature in X.columns:
            mean_col = rolling_by_athlete(X, feature, 7, np.mean, min_periods=3)
            std_col = rolling_by_athlete(X, feature, 7, np.std, min_periods=3)
            X[f'{feature}_cv_7d'] = safe_ratio(std_col, mean_col)
    
    # Athlete-specific z-scores
    for feature in ['actual_tss', 'hrv', 'resting_hr_daily', 'sleep_hours']:
        if feature in X.columns:
            athlete_means = X.groupby('athlete_id')[feature].transform('mean')
            athlete_stds = X.groupby('athlete_id')[feature].transform('std')
            X[f'{feature}_z_score'] = np.where(athlete_stds > 0,
                                               (X[feature] - athlete_means) / athlete_stds,
                                               0)
    
    # ===== Accumulated Risk Features =====
    X['cumulative_risk_7d'] = rolling_by_athlete(X, 'acute_risk_estimate', 7, np.sum)
    
    # Flag athletes with consistently elevated risk
    X['persistently_elevated_risk'] = (
        X.groupby('athlete_id')['acute_risk_estimate']
        .transform(lambda x: (x > 0.4).rolling(window=7, min_periods=5).mean() > 0.7)
    ).astype(int)

    # ===== Critical 7-day Window Features =====
    
    # Recovery metrics to track for the critical window
    recovery_metrics = [
        'deep_sleep', 'rem_sleep', 'hrv', 'resting_hr_daily', 
        'stress', 'body_battery_morning', 'sleep_quality_daily'
    ]
    
    # Create baseline values for each athlete and metric
    for metric in recovery_metrics:
        if metric in X.columns:
            # Calculate personalized baseline for each metric (using non-critical periods)
            X[f'{metric}_baseline'] = X.groupby('athlete_id')[metric].transform(
                lambda x: x.rolling(window=28, min_periods=14).mean().shift(7)
            )
            
            # Calculate deviation from baseline
            X[f'{metric}_deviation'] = X[metric] - X[f'{metric}_baseline']
            
            # Calculate z-score from personalized baseline with protection against zero std
            X[f'{metric}_zscore'] = X.groupby('athlete_id')[metric].transform(
                lambda x: (x - x.rolling(window=28, min_periods=14).mean().shift(7)) /
                        np.maximum(x.rolling(window=28, min_periods=14).std().shift(7), 0.0001)
            ).fillna(0)
            
            # Track days below personal baseline threshold
            X[f'{metric}_below_baseline'] = (X[f'{metric}_zscore'] < -0.5).astype(int)
            X[f'{metric}_consecutive_below'] = consecutive_day_counter(X, f'{metric}_below_baseline')
            
            # Create severe deviation flags
            X[f'{metric}_moderate_deviation'] = (X[f'{metric}_zscore'] < -1.0).astype(int)
            X[f'{metric}_severe_deviation'] = (X[f'{metric}_zscore'] < -2.0).astype(int)
            
            # Calculate 7-day metric trend (to capture critical window slope)
            X[f'{metric}_7d_trend'] = rolling_by_athlete(
                X, f'{metric}_zscore', 7,
                lambda y: np.polyfit(range(len(y)), y, 1)[0] if len(y) > 3 else np.nan,
                min_periods=4
            )
            
            # Count metrics deviating in critical window
            X[f'{metric}_critical_window_sum'] = rolling_by_athlete(
                X, f'{metric}_below_baseline', 7, sum, min_periods=1
            )
    
    # Count total metrics showing deviation in critical window
    deviation_columns = [f'{m}_below_baseline' for m in recovery_metrics if f'{m}_below_baseline' in X.columns]
    if deviation_columns:
        X['total_metrics_deviating'] = X[deviation_columns].sum(axis=1)
        X['critical_window_alert'] = (X['total_metrics_deviating'] >= 3).astype(int)
    
    # ===== Stress-Specific Enhancements =====
    
    if 'stress' in X.columns:
        # Stress volatility
        X['stress_volatility'] = rolling_by_athlete(X, 'stress', 7, np.std, min_periods=3)
        X['stress_volatility_risk'] = X['stress_volatility'] / X['stress_baseline']
        
        # Stress rate of change features
        X['stress_change_1d'] = X.groupby('athlete_id')['stress'].diff()
        X['stress_change_3d'] = X.groupby('athlete_id')['stress'].diff(3)
        X['stress_change_7d'] = X.groupby('athlete_id')['stress'].diff(7)
        X['stress_acceleration'] = X.groupby('athlete_id')['stress_change_1d'].diff()
        
        # Stress exponential growth detection
        X['stress_exp_growth'] = np.where(
            (X['stress_change_1d'] > 0) & 
            (X['stress_change_1d'] > X['stress_change_1d'].shift(1)) &
            (X['stress_change_1d'].shift(1) > X['stress_change_1d'].shift(2)),
            1, 0
        )
        
        # Stress to recovery ratios
        if 'sleep_quality_daily' in X.columns:
            X['stress_to_sleep_ratio'] = safe_ratio(X['stress'], X['sleep_quality_daily'], lower_clip=0.1)
            X['stress_to_sleep_z'] = X.groupby('athlete_id')['stress_to_sleep_ratio'].transform(
                lambda x: (x - x.mean()) / x.std() if x.std() > 0 else 0
            )
        
        if 'hrv' in X.columns:
            X['stress_to_hrv_ratio'] = safe_ratio(X['stress'], X['hrv'], lower_clip=0.01)
            X['stress_to_hrv_z'] = X.groupby('athlete_id')['stress_to_hrv_ratio'].transform(
                lambda x: (x - x.mean()) / x.std() if x.std() > 0 else 0
            )
        
        # High-stress day tracking
        X['high_stress_day'] = (X['stress_zscore'] > 1.0).astype(int)
        X['consecutive_high_stress'] = consecutive_day_counter(X, 'high_stress_day')
        
        # Stress composite 
        X['stress_risk_score'] = (
            (X['stress_zscore'].clip(lower=0) * 0.4) +
            (X['stress_volatility_risk'].clip(lower=0, upper=5) * 0.3) +
            (X['consecutive_high_stress'].clip(upper=7) / 7 * 0.3)
        )
    
    return X

X = engineer_features(X)

/var/folders/lx/jx57qljj5n74_nkl5qm8bqcr0000gn/T/ipykernel_1574/4224841697.py:254: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  X[f'{metric}_baseline'] = X.groupby('athlete_id')[metric].transform(
/var/folders/lx/jx57qljj5n74_nkl5qm8bqcr0000gn/T/ipykernel_1574/4224841697.py:259: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  X[f'{metric}_deviation'] = X[metric] - X[f'{metric}_baseline']
/var/folders/lx/jx57qljj5n74_nkl5qm8bqcr0000gn/T/ipykernel_1574/4224841697.py:262: PerformanceWarning: DataFrame is highly fragmented.  This is 

It is also noticeable that there are 311 days with an actual TSS greater or equal 400 which is extremely high. The planned_TSS on those days is also accordingly high. 29 (9.3%) out of these days are labelled with injury. These days appear to be special training events as they are planned and their correlation with injuries indicates they could be important signals rather than errors.

We have to carefully handle these outliers:
- preserve the original data while adding transformed versions (tree-based models like XGBoost and random forest can work with the original data while LASSO is sensitive to outliers.)
- create categorical markers for extreme days
- track post-extreme recovery periods

In [271]:
# Create categorical features for extreme TSS events 
X['extreme_tss'] = pd.cut(
    X['actual_tss'],
    bins=[0, 300, 400, 500, np.inf],
    labels=['Normal', 'High', 'Very High', 'Extreme']
)


# Create post-extreme monitoring features 
# First create a binary indicator for extreme events
X['is_extreme_event'] = X['extreme_tss'].isin(['Very High', 'Extreme']).astype(int)

# For each athlete, create a date-based tracking system
X = X.sort_values(['athlete_id', 'date'])

# Initialize counter for days since extreme TSS (999 impossible value)
X['days_since_extreme_tss'] = 999

# Process each athlete group to calculate days since extreme event
for athlete, group in X.groupby('athlete_id'):
    # Find indices of extreme events
    extreme_indices = group[group['is_extreme_event'] == 1].index
    
    if len(extreme_indices) > 0:
        # For each day, find the most recent extreme event
        for idx in group.index:
            # Find all extreme events that happened before this day
            previous_extremes = extreme_indices[extreme_indices <= idx]
            if len(previous_extremes) > 0:
                # Get the most recent extreme event index
                last_extreme = previous_extremes[-1]
                # Calculate days difference (this assumes daily entries)
                days_diff = group.loc[idx:idx, 'date'].reset_index(drop=True)[0] - group.loc[last_extreme:last_extreme, 'date'].reset_index(drop=True)[0]
                X.loc[idx, 'days_since_extreme_tss'] = days_diff.days
    
# Mark post-extreme period (7 days after an extreme event)
X['post_extreme_period'] = ((X['days_since_extreme_tss'] <= 7) & 
                                (X['days_since_extreme_tss'].notna())).astype(int)

# Add specific recovery metrics for post-extreme periods
X['post_extreme_hrv_deficit'] = X['post_extreme_period'] * (1 - X['hrv_ratio'])
X['post_extreme_sleep_deficit'] = X['post_extreme_period'] * X['poor_sleep']


/var/folders/lx/jx57qljj5n74_nkl5qm8bqcr0000gn/T/ipykernel_1574/250282501.py:2: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  X['extreme_tss'] = pd.cut(
/var/folders/lx/jx57qljj5n74_nkl5qm8bqcr0000gn/T/ipykernel_1574/250282501.py:11: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  X['is_extreme_event'] = X['extreme_tss'].isin(['Very High', 'Extreme']).astype(int)


Now the lifestyle factors are no longer needed and could lead to overfitting or noise since they are static throughout the year. Let's drop them.

In [272]:
static_features = ['lifestyle', 'hrv_baseline', 'resting_hr_norm', 'max_hr', 'lthr', 'vo2max', 'running_threshold_pace', 'ftp', 'css', 'training_experience', 'weekly_training_hours', 'recovery_rate',
                   'sleep_time_norm', 'sleep_quality_norm', 'stress_factor', 'smoking_factor', 'drinking_factor', 'nutrition_factor', 'age', 'height_cm', 'weight_kg', 'genetic_factor', 'gender', 'extreme_tss']

X.drop(static_features, axis=1, inplace=True)

X.head()

,athlete_id,date,resting_hr_daily,hrv,sleep_hours,deep_sleep,light_sleep,rem_sleep,sleep_quality_daily,body_battery_morning,stress,body_battery_evening,planned_tss,actual_tss,swim_duration_minutes,swim_tss,swim_intensity_factor,swim_distance_km,swim_avg_speed_kph,swim_avg_hr,swim_max_hr,bike_duration_minutes,bike_tss,bike_intensity_factor,bike_distance_km,bike_avg_speed_kph,bike_avg_hr,bike_max_hr,bike_avg_power,bike_normalized_power,bike_work_kilojoules,bike_elevation_gain,run_duration_minutes,run_tss,run_intensity_factor,run_distance_km,run_avg_speed_kph,run_avg_hr,run_max_hr,run_elevation_gain,strength_duration_minutes,strength_tss,strength_intensity_factor,strength_avg_hr,strength_max_hr,bike_hr_zones_Z1,bike_hr_zones_Z2,bike_hr_zones_Z3,bike_hr_zones_Z4,bike_hr_zones_Z5,bike_hr_zones_Z6,run_hr_zones_Z1,run_hr_zones_Z2,run_hr_zones_Z3,run_hr_zones_Z4,run_hr_zones_Z5,run_hr_zones_Z6,bike_power_zones_Z1,bike_power_zones_Z2,bike_power_zones_Z3,bike_power_zones_Z4,bike_power_zones_Z5,bike_power_zones_Z6,bike_power_zones_Z7,swim_hr_zones_Z1,swim_hr_zones_Z2,swim_hr_zones_Z3,swim_hr_zones_Z4,swim_hr_zones_Z5,swim_hr_zones_Z6,strength_hr_zones_Z1,strength_hr_zones_Z2,strength_hr_zones_Z3,strength_hr_zones_Z4,strength_hr_zones_Z5,strength_hr_zones_Z6,hrv_ratio,hrv_zone,low_hrv_risk,consecutive_low_hrv,hrv_7d_slope,hrv_volatility,hrv_volatility_risk,resting_hr_ratio,rhr_risk,poor_sleep,sleep_debt,sleep_hours_risk,tss_deviation,tss_deviation_pct,high_tss_day,consecutive_high_tss,training_adherence,tss_7d_avg,acute_load,chronic_load,acwr,acwr_risk,acwr_volatility,acwr_volatility_risk,tss_7d_std,training_monotony,training_strain,previous_week_load,week_to_week_change,ramp_rate_risk,chronic_load_42d,ewma_acute,ewma_chronic,ewma_acwr,load_recovery_interaction,sleep_hrv_compound,recovery_composite,acute_risk_estimate,high_risk_multiplier,tss_relative_change,tss_spike,tss_3d_mean,tss_prev_10d_mean,taper_spike_pattern,recovery_adequacy,sleep_to_load_ratio,recovery_window_after_high_load,acwr_squared,acwr_hrv_interaction,acwr_sleep_interaction,combined_risk_interaction,acute_load_change_1d,acute_load_change_7d,acute_load_acceleration,chronic_load_change_1d,chronic_load_change_7d,chronic_load_acceleration,hrv_ratio_change_1d,hrv_ratio_change_7d,hrv_ratio_acceleration,sleep_quality_daily_change_1d,sleep_quality_daily_change_7d,sleep_quality_daily_acceleration,actual_tss_cv_7d,hrv_cv_7d,sleep_hours_cv_7d,actual_tss_z_score,hrv_z_score,resting_hr_daily_z_score,sleep_hours_z_score,cumulative_risk_7d,persistently_elevated_risk,deep_sleep_baseline,deep_sleep_deviation,deep_sleep_zscore,deep_sleep_below_baseline,deep_sleep_consecutive_below,deep_sleep_moderate_deviation,deep_sleep_severe_deviation,deep_sleep_7d_trend,deep_sleep_critical_window_sum,rem_sleep_baseline,rem_sleep_deviation,rem_sleep_zscore,rem_sleep_below_baseline,rem_sleep_consecutive_below,rem_sleep_moderate_deviation,rem_sleep_severe_deviation,rem_sleep_7d_trend,rem_sleep_critical_window_sum,hrv_deviation,hrv_zscore,hrv_below_baseline,hrv_consecutive_below,hrv_moderate_deviation,hrv_severe_deviation,hrv_7d_trend,hrv_critical_window_sum,resting_hr_daily_baseline,resting_hr_daily_deviation,resting_hr_daily_zscore,resting_hr_daily_below_baseline,resting_hr_daily_consecutive_below,resting_hr_daily_moderate_deviation,resting_hr_daily_severe_deviation,resting_hr_daily_7d_trend,resting_hr_daily_critical_window_sum,stress_baseline,stress_deviation,stress_zscore,stress_below_baseline,stress_consecutive_below,stress_moderate_deviation,stress_severe_deviation,stress_7d_trend,stress_critical_window_sum,body_battery_morning_baseline,body_battery_morning_deviation,body_battery_morning_zscore,body_battery_morning_below_baseline,body_battery_morning_consecutive_below,body_battery_morning_moderate_deviation,body_battery_morning_severe_deviation,body_battery_morning_7d_trend,body_battery_morning_critical_window_sum,sleep_quality_daily_baseline,sleep_quality_daily_deviation,sleep_quality_daily_zscore,

## Encoding Categorical Features

Many machine learning models require all input features to be numeric. Therefore, categorical features must be transformed before model training. Below, we apply appropriate encoding techniques based on feature types:

- **Ordinal Encoding** for features with a meaningful order
- **One-Hot Encoding** for nominal features with no inherent order
- **Label Encoding** for tree-based models or features with minimal categories

In [273]:
# Identify categorical columns and their types
categorical_columns = {
    # Binary categorical
    'binary': ['gender'],
    
    # Nominal categorical (no inherent order)
    'nominal': ['lifestyle'],
    
    # Ordinal categorical (clear order)
    'ordinal': {
        'hrv_zone': ['below normal', 'normal', 'above normal'],
        'training_adherence': ['undertraining', 'on target', 'overtraining'],
        'acwr_risk': ['too low', 'optimal', 'danger zone', 'high risk'],
        'ramp_rate_risk': ['decreasing', 'optimal', 'warning', 'high risk'],
        'extreme_tss': ['normal', 'high', 'very high', 'extreme']
    }
}

def encode_categorical_features(X):
    """
    Handle categorical features appropriately for tree-based models.
    
    Args:
        X: Pandas DataFrame containing the features
        
    Returns:
        X_encoded: DataFrame with encoded categorical features
        feature_names: List of feature names after encoding
    """

    # Make sure categorical values are strings first, then lowercase
    for cat_type in categorical_columns:
        if cat_type == 'ordinal':
            for col in categorical_columns[cat_type]:
                if col in X.columns:
                    X[col] = X[col].astype(str).str.lower()
        else:
            for col in categorical_columns[cat_type]:
                if col in X.columns:
                    X[col] = X[col].astype(str).str.lower()

    # Get all categorical column names
    all_cat_cols = (
        categorical_columns['binary'] + 
        categorical_columns['nominal'] + 
        list(categorical_columns['ordinal'].keys())
    )
    
    # Filter to only include columns that exist in the dataset
    all_cat_cols = [col for col in all_cat_cols if col in X.columns]
    
    # Split between ordinal and non-ordinal columns
    ordinal_cols = [col for col in categorical_columns['ordinal'] if col in X.columns]
    nonordinal_cols = [col for col in all_cat_cols if col not in ordinal_cols]
    
    # Create transformer list
    transformers = []
    
    # Add one-hot encoder for nominal features
    if nonordinal_cols:
        transformers.append(
            ('onehot', OneHotEncoder(sparse_output=False, drop='first'), nonordinal_cols)
        )
    
    # Add ordinal encoder for ordinal features
    if ordinal_cols:
        # Create ordinal encoder with proper category ordering
        for col in ordinal_cols:
            categories = [categorical_columns['ordinal'][col]]
            encoder = OrdinalEncoder(categories=categories)
            transformers.append((f'ordinal_{col}', encoder, [col]))
    
    # Create and apply column transformer
    preprocessor = ColumnTransformer(
        transformers=transformers,
        remainder='passthrough'  # Keep other columns as is
    )
    
    # Fit and transform the data
    X_transformed = preprocessor.fit_transform(X)
    
    # Get feature names after transformation
    feature_names = []
    
    # Get feature names from one-hot encoding
    if nonordinal_cols and transformers:
        onehot_idx = [i for i, (name, _, _) in enumerate(transformers) if name == 'onehot'][0]
        onehot_features = preprocessor.transformers_[onehot_idx][1].get_feature_names_out(nonordinal_cols)
        feature_names.extend(onehot_features)
    
    # Add ordinal feature names
    feature_names.extend(ordinal_cols)
    
    # Add remaining feature names that weren't transformed
    non_cat_cols = [col for col in X.columns if col not in all_cat_cols]
    feature_names.extend(non_cat_cols)
    
    # Create a DataFrame with transformed features and proper column names
    X_encoded = pd.DataFrame(X_transformed, columns=feature_names, index=X.index)
    
    return X_encoded, feature_names

X_encoded, feature_names = encode_categorical_features(X)

# drop athlete_id as ML models cannot interpret strings
X_encoded.drop(['athlete_id'], axis=1, inplace=True)

# make sure there are no null values within the newly engineered features
X_encoded.fillna(0, inplace=True)

X_encoded.head()

/var/folders/lx/jx57qljj5n74_nkl5qm8bqcr0000gn/T/ipykernel_1574/3752271000.py:109: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  X_encoded.fillna(0, inplace=True)


,hrv_zone,training_adherence,acwr_risk,ramp_rate_risk,date,resting_hr_daily,hrv,sleep_hours,deep_sleep,light_sleep,rem_sleep,sleep_quality_daily,body_battery_morning,stress,body_battery_evening,planned_tss,actual_tss,swim_duration_minutes,swim_tss,swim_intensity_factor,swim_distance_km,swim_avg_speed_kph,swim_avg_hr,swim_max_hr,bike_duration_minutes,bike_tss,bike_intensity_factor,bike_distance_km,bike_avg_speed_kph,bike_avg_hr,bike_max_hr,bike_avg_power,bike_normalized_power,bike_work_kilojoules,bike_elevation_gain,run_duration_minutes,run_tss,run_intensity_factor,run_distance_km,run_avg_speed_kph,run_avg_hr,run_max_hr,run_elevation_gain,strength_duration_minutes,strength_tss,strength_intensity_factor,strength_avg_hr,strength_max_hr,bike_hr_zones_Z1,bike_hr_zones_Z2,bike_hr_zones_Z3,bike_hr_zones_Z4,bike_hr_zones_Z5,bike_hr_zones_Z6,run_hr_zones_Z1,run_hr_zones_Z2,run_hr_zones_Z3,run_hr_zones_Z4,run_hr_zones_Z5,run_hr_zones_Z6,bike_power_zones_Z1,bike_power_zones_Z2,bike_power_zones_Z3,bike_power_zones_Z4,bike_power_zones_Z5,bike_power_zones_Z6,bike_power_zones_Z7,swim_hr_zones_Z1,swim_hr_zones_Z2,swim_hr_zones_Z3,swim_hr_zones_Z4,swim_hr_zones_Z5,swim_hr_zones_Z6,strength_hr_zones_Z1,strength_hr_zones_Z2,strength_hr_zones_Z3,strength_hr_zones_Z4,strength_hr_zones_Z5,strength_hr_zones_Z6,hrv_ratio,low_hrv_risk,consecutive_low_hrv,hrv_7d_slope,hrv_volatility,hrv_volatility_risk,resting_hr_ratio,rhr_risk,poor_sleep,sleep_debt,sleep_hours_risk,tss_deviation,tss_deviation_pct,high_tss_day,consecutive_high_tss,tss_7d_avg,acute_load,chronic_load,acwr,acwr_volatility,acwr_volatility_risk,tss_7d_std,training_monotony,training_strain,previous_week_load,week_to_week_change,chronic_load_42d,ewma_acute,ewma_chronic,ewma_acwr,load_recovery_interaction,sleep_hrv_compound,recovery_composite,acute_risk_estimate,high_risk_multiplier,tss_relative_change,tss_spike,tss_3d_mean,tss_prev_10d_mean,taper_spike_pattern,recovery_adequacy,sleep_to_load_ratio,recovery_window_after_high_load,acwr_squared,acwr_hrv_interaction,acwr_sleep_interaction,combined_risk_interaction,acute_load_change_1d,acute_load_change_7d,acute_load_acceleration,chronic_load_change_1d,chronic_load_change_7d,chronic_load_acceleration,hrv_ratio_change_1d,hrv_ratio_change_7d,hrv_ratio_acceleration,sleep_quality_daily_change_1d,sleep_quality_daily_change_7d,sleep_quality_daily_acceleration,actual_tss_cv_7d,hrv_cv_7d,sleep_hours_cv_7d,actual_tss_z_score,hrv_z_score,resting_hr_daily_z_score,sleep_hours_z_score,cumulative_risk_7d,persistently_elevated_risk,deep_sleep_baseline,deep_sleep_deviation,deep_sleep_zscore,deep_sleep_below_baseline,deep_sleep_consecutive_below,deep_sleep_moderate_deviation,deep_sleep_severe_deviation,deep_sleep_7d_trend,deep_sleep_critical_window_sum,rem_sleep_baseline,rem_sleep_deviation,rem_sleep_zscore,rem_sleep_below_baseline,rem_sleep_consecutive_below,rem_sleep_moderate_deviation,rem_sleep_severe_deviation,rem_sleep_7d_trend,rem_sleep_critical_window_sum,hrv_deviation,hrv_zscore,hrv_below_baseline,hrv_consecutive_below,hrv_moderate_deviation,hrv_severe_deviation,hrv_7d_trend,hrv_critical_window_sum,resting_hr_daily_baseline,resting_hr_daily_deviation,resting_hr_daily_zscore,resting_hr_daily_below_baseline,resting_hr_daily_consecutive_below,resting_hr_daily_moderate_deviation,resting_hr_daily_severe_deviation,resting_hr_daily_7d_trend,resting_hr_daily_critical_window_sum,stress_baseline,stress_deviation,stress_zscore,stress_below_baseline,stress_consecutive_below,stress_moderate_deviation,stress_severe_deviation,stress_7d_trend,stress_critical_window_sum,body_battery_morning_baseline,body_battery_morning_deviation,body_battery_morning_zscore,body_battery_morning_below_baseline,body_battery_morning_consecutive_below,body_battery_morning_moderate_deviation,body_battery_morning_severe_deviation,body_battery_morning_7d_trend,body_battery_morning_critical_window_sum,sleep_quality_daily_baseline,sleep_quality_daily_deviation,sleep_quality_daily_zscore,sleep_quali

And finally handle dates and transform them into meaningful features, as ML models cannot handle datetime objects.

In [274]:
new_date_features = {}

for col in X_encoded.columns:
    if 'date' in col.lower() or X_encoded[col].dtype == 'datetime64[ns]':
        try:
            dt_col = pd.to_datetime(X_encoded[col])
            new_date_features[f'{col}_dayofweek'] = dt_col.dt.dayofweek
            new_date_features[f'{col}_season'] = (dt_col.dt.month % 12 + 3) // 3
            print(f"Processed date column: {col}")
        except:
            print(f"Couldn't process {col} as a date")

# Convert the dictionary to a DataFrame and concat all new features at once
new_date_df = pd.DataFrame(new_date_features, index=X_encoded.index)

# Drop original date columns
X_encoded = X_encoded.drop(columns=[col for col in X_encoded.columns if col in new_date_df.columns or 'date' in col.lower()])

# Now concat new date features (avoids fragmentation)
X_encoded = pd.concat([X_encoded, new_date_df], axis=1)

Processed date column: date


## Save Processed Data

We export the processed features dataFrame and target variable.

In [275]:
# Save for the next notebook
X_encoded.to_csv('../simulated_data/processed/processed_features.csv', index=False)
y.to_csv('../simulated_data/processed/target_variable.csv', index=False)

## Split data into training and testing sets

Here there are two options:
1. **Split by time period** 
    - use earlier months for training
    - use later months for testing (e.g., last 60 days of the year)
    - keep the same athletes in both training and testing sets but at different time periods

2. **Split by athlete**
    - some athlete's entire data goes to training (e.g., 800 athletes)
    - some athlete's entire data goes to testing (e.g., 200 athletes)

The question guiding the decision between these option is:
- Do we want our model to generize well to completely new athletes or
- Do we want our model to predict future injuries for already known athletes?

In [276]:
# Option 1: time-based split
# Create a time-based cutoff 
cutoff_date = pd.to_datetime('2024-11-01')  

# Create train/test masks
train_mask = merged['date'] < cutoff_date
test_mask = merged['date'] >= cutoff_date

# Split the data
X_train_time_based = X_encoded[train_mask]
X_test_time_based = X_encoded[test_mask] 
y_train_time_based = y[train_mask]
y_test_time_based = y[test_mask]


# Option 2: athlete-based split
# Set random state for reproducibility
random_state = 42

# Create a GroupShuffleSplit to keep all data from one athlete together
splitter = GroupShuffleSplit(n_splits=1, test_size=0.2, random_state=random_state)

# Get the train/test indices based on athlete_id groups
train_idx, test_idx = next(splitter.split(merged, groups=merged['athlete_id']))

# Create masks for indexing
train_mask = np.zeros(len(merged), dtype=bool)
train_mask[train_idx] = True
test_mask = ~train_mask

# Split processed features and target
X_train_athlete_based = X_encoded.iloc[train_idx]
X_test_athlete_based = X_encoded.iloc[test_idx]
y_train_athlete_based = y.iloc[train_idx]
y_test_athlete_based = y.iloc[test_idx]

# check how many athletes are in each set
train_athletes = merged.iloc[train_idx]['athlete_id'].nunique()
test_athletes = merged.iloc[test_idx]['athlete_id'].nunique()
print(f"Training set: {train_athletes} athletes")
print(f"Testing set: {test_athletes} athletes")

# Save for the next notebook
X_train_time_based.to_csv('../simulated_data/processed/time_based/X_train.csv', index=False)
X_test_time_based.to_csv('../simulated_data/processed/time_based/X_test.csv', index=False)
y_train_time_based.to_csv('../simulated_data/processed/time_based/y_train.csv', index=False)
y_test_time_based.to_csv('../simulated_data/processed/time_based/y_test.csv', index=False)

X_train_athlete_based.to_csv('../simulated_data/processed/athlete_based/X_train.csv', index=False)
X_test_athlete_based.to_csv('../simulated_data/processed/athlete_based/X_test.csv', index=False)
y_train_athlete_based.to_csv('../simulated_data/processed/athlete_based/y_train.csv', index=False)
y_test_athlete_based.to_csv('../simulated_data/processed/athlete_based/y_test.csv', index=False)

/var/folders/lx/jx57qljj5n74_nkl5qm8bqcr0000gn/T/ipykernel_1574/3272406523.py:10: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  X_train_time_based = X_encoded[train_mask]
/var/folders/lx/jx57qljj5n74_nkl5qm8bqcr0000gn/T/ipykernel_1574/3272406523.py:11: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  X_test_time_based = X_encoded[test_mask]


Training set: 800 athletes
Testing set: 200 athletes
